In [1]:
import pandas as pd
import numpy as np
import nltk
from collections import defaultdict

### Import Scripts

In [195]:
full_episode_df = pd.read_csv("../05_Data/04_Model_Output/181113_full_episodes_by_topic.csv",index_col=0)
full_episode_df["season"] = full_episode_df["season_ep"].apply(lambda x: x[:3])
full_episode_df.head(5)

,season_ep,topic,score,Finance/Organized Crime,Medical Malpractice,Infant/Baby Welfare,Sexual Assault,Bombing/Terrorism,Russian Mob,Political Corruption,Murder Defense,Military,Judiasm,Child Welfare,Suicide,School/University,Infectious Diseases,Religion,Drugs/Gangs/Robbery,season
0,s01e01,Medical Malpractice,0.380267,0.000000,0.380267,0.000000,0.003791,0.000000,0.001068,0.000000,0.000000,0.012291,0.000000,0.000000,0.000000,0.019285,0.038327,0.000000,0.008754,s01
1,s01e02,Drugs/Gangs/Robbery,0.139380,0.000000,0.005877,0.024975,0.124810,0.008524,0.000000,0.000000,0.041899,0.000000,0.009996,0.000000,0.011568,0.000000,0.015861,0.001081,0.139380,s01
2,s01e03,Suicide,0.328722,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.328722,0.000000,0.227104,0.011089,0.007953,s01
3,s01e04,Political Corruption,0.107163,0.022007,0.001738,0.000000,0.069170,0.000000,0.000844,0.107163,0.038305,0.000000,0.000000,0.032959,0.000000,0.026432,0.058988,0.025017,0.000000,s01
4,s01e05,Drugs/Gangs/Robbery,0.157249,0.018024,0.000000,0.013192,0.000000,0.000000,0.000000,0.048168,0.015766,0.000000,0.000000,0.000000,0.000000,0.014938,0.005044,0.022405,0.157249,s01


## Full Episodes

### Aggregate Topics by Episodes

In [193]:
topic_list = full_episode_df.columns.tolist()[3:-1]  # list of topics

ep_topic_df = full_episode_df.groupby(["season","topic"])[["score"]].count()
ep_topic_df = ep_topic_df.reset_index()
ep_topic_df = ep_topic_df.reindex(columns=(ep_topic_df.columns.tolist() + topic_list))

for index, row in ep_topic_df.iterrows():
    ep_topic = ep_topic_df.iloc[index]["topic"]
    ep_score = ep_topic_df.iloc[index]["score"]
    ep_topic_df[ep_topic][index] = ep_score
    
ep_topic_df = ep_topic_df.fillna(0)
ep_topic_df = ep_topic_df.drop(["topic","score"],axis=1)
ep_topic_df = ep_topic_df.groupby("season").sum().reset_index()
ep_topic_df.head(10)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


,season,Finance/Organized Crime,Medical Malpractice,Infant/Baby Welfare,Sexual Assault,Bombing/Terrorism,Russian Mob,Political Corruption,Murder Defense,Military,Judiasm,Child Welfare,Suicide,School/University,Infectious Diseases,Religion,Drugs/Gangs/Robbery
0,s01,0.0,2.0,0.0,2.0,1.0,2.0,2.0,0.0,1.0,0.0,0.0,2.0,0.0,2.0,1.0,7.0
1,s02,2.0,1.0,2.0,2.0,0.0,0.0,3.0,3.0,0.0,0.0,1.0,1.0,1.0,0.0,3.0,3.0
2,s03,3.0,2.0,0.0,1.0,0.0,0.0,1.0,1.0,2.0,2.0,1.0,0.0,2.0,1.0,1.0,5.0
3,s04,2.0,1.0,1.0,1.0,1.0,2.0,1.0,3.0,0.0,1.0,3.0,1.0,0.0,1.0,1.0,3.0
4,s05,3.0,4.0,2.0,2.0,1.0,0.0,3.0,1.0,0.0,0.0,2.0,0.0,2.0,1.0,1.0,1.0
5,s06,5.0,1.0,2.0,0.0,1.0,0.0,3.0,2.0,0.0,1.0,3.0,0.0,4.0,0.0,0.0,1.0
6,s07,5.0,1.0,0.0,3.0,0.0,0.0,5.0,0.0,0.0,0.0,1.0,0.0,2.0,0.0,1.0,5.0
7,s08,0.0,4.0,2.0,4.0,0.0,0.0,3.0,3.0,2.0,0.0,0.0,0.0,0.0,1.0,2.0,3.0
8,s09,0.0,1.0,2.0,0.0,0.0,2.0,3.0,1.0,0.0,1.0,1.0,1.0,3.0,1.0,1.0,7.0
9,s10,3.0,1.0,1.0,3.0,1.0,0.0,3.0,3.0,1.0,0.0,1.0,2.0,0.0,1.0,0.0,4.0


### Aggregate Topics by Partial Episodes

In [226]:
fractional_topic_df = full_episode_df.groupby(["season"]).sum().reset_index().drop(["score"],axis=1)
fractional_topic_df = fractional_topic_df.set_index("season")
fractional_topic_df.head()

,Finance/Organized Crime,Medical Malpractice,Infant/Baby Welfare,Sexual Assault,Bombing/Terrorism,Russian Mob,Political Corruption,Murder Defense,Military,Judiasm,Child Welfare,Suicide,School/University,Infectious Diseases,Religion,Drugs/Gangs/Robbery
season,,,,,,,,,,,,,,,,
s01,0.400874,0.770808,0.444962,1.143568,0.492110,0.533049,0.413753,0.166799,0.442949,0.072969,0.188456,0.589582,0.212694,0.913001,0.594239,1.644614
s02,0.866623,0.451246,0.498893,0.880079,0.067710,0.075424,0.649591,0.822424,0.152680,0.091490,0.575150,0.629846,0.400176,0.287129,0.815441,0.969085
s03,0.809739,1.232667,0.125064,0.656599,0.077845,0.098694,0.438966,0.587795,0.396959,0.710827,0.340554,0.312635,0.554051,0.502593,0.324668,1.155143
s04,0.694921,0.451315,0.812759,0.459880,0.532935,0.541911,0.549315,0.725856,0.072088,0.592828,0.908082,0.185960,0.513596,0.407053,0.751652,0.941985
s05,0.765812,1.358713,0.976558,0.950909,0.444637,0.045833,0.615624,0.550260,0.251289,0.169698,0.645852,0.281074,0.676074,0.384396,0.403208,0.978852


In [227]:
transpose_topic_df = fractional_topic_df.transpose()
transpose_topic_df.head()

season,s01,s02,s03,s04,s05,s06,s07,s08,s09,s10,s11,s12,s13,s14,s15,s16,s17,s18,s19,s20
Finance/Organized Crime,0.400874,0.866623,0.809739,0.694921,0.765812,1.131520,0.985403,0.316366,0.476548,0.870747,0.576416,0.836812,1.276085,0.959321,0.298293,0.291956,0.261745,0.167898,0.474933,0.312730
Medical Malpractice,0.770808,0.451246,1.232667,0.451315,1.358713,0.266942,0.566752,1.149528,0.933285,0.483380,0.414490,0.652546,0.732820,0.553298,0.617332,0.311889,0.369625,0.625376,0.127945,0.625673
Infant/Baby Welfare,0.444962,0.498893,0.125064,0.812759,0.976558,0.978194,0.123874,0.934073,0.691056,0.590564,0.456249,0.256675,0.381928,0.555786,0.114628,0.931160,0.463900,0.485127,0.431489,0.584305
Sexual Assault,1.143568,0.880079,0.656599,0.459880,0.950909,0.627415,0.917199,0.919868,0.387099,0.979904,0.201884,0.139716,0.216246,0.154132,0.191337,0.455565,0.658328,0.190383,0.317022,0.206815
Bombing/Terrorism,0.492110,0.067710,0.077845,0.532935,0.444637,0.278835,0.074734,0.041672,0.093448,0.297859,0.029833,0.072999,0.064275,0.016859,0.118373,0.516773,0.276400,0.268650,0.270566,0.620606


In [228]:
transpose_topic_df.to_csv("../05_Data/05_Graphics/full_episode_topics_by_season.csv")

## First Half of Episode

In [196]:
half_1st_episode_df = pd.read_csv("../05_Data/04_Model_Output/181113_1st_half_episodes_by_topic.csv",index_col=0)
half_1st_episode_df["season"] = half_1st_episode_df["season_ep"].apply(lambda x: x[:3])
half_1st_episode_df.head(5)

,season_ep,topic,score,Bar/Drinking,Shooting/Guns,Sexual Assault,Vehicle,School,Hospital,Email/Phone/Mail,Tourist/Visitor,Russian Mob,Party/Nightclub,Apartment Building,Bombing,Credit Card Purchases,Arson,Money,Romantic Relationship,season
0,s01e01,Hospital,0.316450,0.000000,0.004828,0.000000,0.022824,0.009154,0.316450,0.003343,0.000000,0.000000,0.014423,0.000000,0.002525,0.000000,0.000000,0.0,0.000000,s01
1,s01e02,Shooting/Guns,0.117147,0.040947,0.117147,0.026128,0.020847,0.037297,0.023944,0.008812,0.000000,0.001245,0.000000,0.000000,0.016842,0.000000,0.052076,0.0,0.027333,s01
2,s01e03,Arson,0.136869,0.025578,0.098967,0.001503,0.013431,0.093121,0.000000,0.000000,0.003430,0.000000,0.045636,0.002848,0.008999,0.029815,0.136869,0.0,0.000000,s01
3,s01e04,Party/Nightclub,0.148483,0.114739,0.000000,0.001157,0.000000,0.027437,0.004117,0.000000,0.018604,0.000000,0.148483,0.000000,0.000000,0.000000,0.000000,0.0,0.000237,s01
4,s01e05,Shooting/Guns,0.213549,0.011183,0.213549,0.005704,0.000000,0.000000,0.000000,0.000000,0.037914,0.000445,0.000000,0.000000,0.000000,0.164093,0.000000,0.0,0.000000,s01


### Aggregate Topics by Partial Episodes

In [224]:
fractional_topic_1st_df = half_1st_episode_df.groupby(["season"]).sum().reset_index().drop(["score"],axis=1)
fractional_topic_1st_df = fractional_topic_1st_df.set_index("season")
fractional_topic_1st_df.head(5)

,Bar/Drinking,Shooting/Guns,Sexual Assault,Vehicle,School,Hospital,Email/Phone/Mail,Tourist/Visitor,Russian Mob,Party/Nightclub,Apartment Building,Bombing,Credit Card Purchases,Arson,Money,Romantic Relationship
season,,,,,,,,,,,,,,,,
s01,0.609123,1.995729,1.068930,0.414596,0.271998,0.810453,0.187019,0.162472,0.512068,0.369626,0.507052,0.791329,0.306717,0.639744,0.114483,0.176942
s02,0.924327,0.730313,0.725741,0.175258,0.474944,0.874364,0.484510,0.140846,0.182910,0.575511,1.227344,0.223522,0.287403,0.332968,0.187414,0.581354
s03,0.835552,1.139164,0.169012,0.412231,0.533019,1.000545,0.515671,0.346583,0.248605,0.672704,0.500786,0.219079,0.858686,0.248164,0.083522,0.545811
s04,0.807709,0.935211,0.292888,0.888020,0.397894,0.758455,0.317982,0.661869,0.618095,0.249992,0.445353,0.802532,0.243231,0.571143,0.276267,0.598315
s05,0.673947,0.707378,0.781864,0.450634,0.456503,1.363248,0.311329,0.480225,0.053868,0.687815,0.474482,0.700545,0.672083,0.340769,0.236712,1.119456


In [225]:
transpse_topic_1st_df = fractional_topic_1st_df.transpose()
transpse_topic_1st_df.head()

season,s01,s02,s03,s04,s05,s06,s07,s08,s09,s10,s11,s12,s13,s14,s15,s16,s17,s18,s19,s20
Bar/Drinking,0.609123,0.924327,0.835552,0.807709,0.673947,1.145749,0.397850,0.232362,0.553986,0.299941,0.405453,0.631001,1.830060,1.317884,0.570178,0.552502,0.272176,0.277454,0.484288,0.315814
Shooting/Guns,1.995729,0.730313,1.139164,0.935211,0.707378,0.474766,0.574468,1.035554,0.656617,0.553055,0.768882,0.588628,0.346851,0.469108,0.833552,0.948038,0.833411,0.558167,0.240788,0.424409
Sexual Assault,1.068930,0.725741,0.169012,0.292888,0.781864,0.394737,0.260632,0.931917,0.318050,0.431382,0.243152,0.050669,0.160127,0.195834,0.217055,0.358981,0.161858,0.072102,0.492688,0.190603
Vehicle,0.414596,0.175258,0.412231,0.888020,0.450634,0.951242,0.868087,1.046998,1.261194,0.901882,1.047589,0.644009,0.554144,0.878176,0.695346,1.054899,0.632810,0.635213,0.774983,0.435719
School,0.271998,0.474944,0.533019,0.397894,0.456503,1.220631,0.646225,0.610437,0.862143,0.698779,0.770890,0.916454,0.587995,0.534552,0.445082,0.585634,0.531279,0.397176,0.373925,0.927208


In [229]:
transpse_topic_1st_df.to_csv("../05_Data/05_Graphics/1st_half_topics_by_season.csv")

## Second Half of Episode

In [201]:
half_2nd_episode_df = pd.read_csv("../05_Data/04_Model_Output/181113_2nd_half_episodes_by_topic.csv",index_col=0)
half_2nd_episode_df["season"] = half_2nd_episode_df["season_ep"].apply(lambda x: x[:3])
half_2nd_episode_df.head(5)

,season_ep,topic,score,Lying Witness,Plead Mental Illness,Legal/Religious Privilege,Video/Audio Evidence,Military Personel,Bank Records,Plea Deal,Racism,Religious Beliefs,NY Governor,AIDS,Websites/Online Behavior,Government Grievances,Appeal Verdict,Internal Police Affairs,Prison,season
0,s01e01,Legal/Religious Privilege,0.076611,0.062903,0.037253,0.076611,0.000000,0.000000,0.000000,0.003954,0.000000,0.000000,0.0,0.064699,0.000000,0.067244,0.00000,0.000000,0.000000,s01
1,s01e02,Prison,0.149230,0.028742,0.000000,0.002842,0.011142,0.010861,0.000000,0.013107,0.107201,0.010512,0.0,0.012758,0.016451,0.005677,0.00000,0.069250,0.149230,s01
2,s01e03,AIDS,0.576181,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.013061,0.000000,0.000000,0.0,0.576181,0.000000,0.000000,0.00000,0.000000,0.000000,s01
3,s01e04,Plea Deal,0.118947,0.042310,0.027490,0.000000,0.000000,0.000251,0.002526,0.118947,0.009728,0.011485,0.0,0.019729,0.003435,0.047395,0.00849,0.002341,0.026162,s01
4,s01e05,Plea Deal,0.160090,0.014835,0.002864,0.000000,0.000000,0.000000,0.000000,0.160090,0.022448,0.000000,0.0,0.002069,0.002140,0.000000,0.00000,0.112904,0.000000,s01


### Aggregate Topics by Partial Episodes

In [222]:
fractional_topic_2nd_df = half_2nd_episode_df.groupby(["season"]).sum().reset_index().drop(["score"],axis=1)
fractional_topic_2nd_df.set_index("season",inplace=True)
fractional_topic_2nd_df.head(5)

,Lying Witness,Plead Mental Illness,Legal/Religious Privilege,Video/Audio Evidence,Military Personel,Bank Records,Plea Deal,Racism,Religious Beliefs,NY Governor,AIDS,Websites/Online Behavior,Government Grievances,Appeal Verdict,Internal Police Affairs,Prison
season,,,,,,,,,,,,,,,,
s01,0.541644,0.229811,0.426961,0.340513,0.270261,0.446743,2.511693,0.373961,0.278308,0.121903,1.360866,0.138759,0.612126,0.280326,0.524413,0.371056
s02,0.344723,0.759444,0.271607,1.191422,0.069656,0.676119,1.623431,0.349009,0.535066,0.201190,0.177402,0.193695,0.476765,1.637498,0.480037,0.294931
s03,0.436214,1.184304,0.520116,0.744809,0.254956,0.705636,1.031365,0.639840,0.128884,0.028898,0.116920,0.171404,0.608276,0.845564,1.269690,0.297852
s04,0.826446,0.840669,0.564384,0.322870,0.083006,0.403023,1.233061,1.211642,0.716701,0.049106,0.090775,0.122725,0.412227,1.055147,0.194344,0.209567
s05,0.817031,0.825712,0.728881,0.838318,0.315084,0.653158,1.101530,0.839869,0.265160,0.048334,0.459308,0.125621,0.400806,0.731242,0.507551,0.686101


In [223]:
transpose_topic_2nd_df = fractional_topic_2nd_df.transpose()
transpose_topic_2nd_df.head()

season,s01,s02,s03,s04,s05,s06,s07,s08,s09,s10,s11,s12,s13,s14,s15,s16,s17,s18,s19,s20
Lying Witness,0.541644,0.344723,0.436214,0.826446,0.817031,1.207721,0.995339,0.318925,0.417297,0.818594,0.770354,0.715355,0.955712,0.855199,0.693206,0.722109,0.545613,0.504482,1.086774,0.938231
Plead Mental Illness,0.229811,0.759444,1.184304,0.840669,0.825712,0.826195,0.306444,0.982783,0.685049,1.316593,1.112211,0.539313,0.950341,0.531264,0.795701,0.443982,0.474881,0.236603,0.349384,0.422334
Legal/Religious Privilege,0.426961,0.271607,0.520116,0.564384,0.728881,0.559285,0.910914,0.947353,0.822918,0.689648,0.837830,1.225042,0.281008,0.331492,0.527996,0.540124,0.461262,0.205297,0.368253,0.650837
Video/Audio Evidence,0.340513,1.191422,0.744809,0.322870,0.838318,0.128596,0.461297,0.675734,0.059715,0.424269,0.603191,0.650041,0.335594,0.379938,0.329600,0.366324,0.900412,1.025371,0.088850,0.279655
Military Personel,0.270261,0.069656,0.254956,0.083006,0.315084,0.061966,0.076282,0.450169,0.327699,0.442272,0.206605,1.050984,0.123559,1.063446,0.948015,0.392263,0.691318,0.385590,0.340664,0.709841


In [230]:
transpose_topic_2nd_df.to_csv("../05_Data/05_Graphics/2nd_half_topics_by_season.csv")